In [1]:
import sys
sys.path.append('a/UltimeTradingBot/Crypto_backtest_tools')
from utilities.get_data import get_historical_from_db
from utilities.backtesting import basic_single_asset_backtest, plot_wallet_vs_asset, get_metrics
import pandas as pd
import ccxt
import matplotlib.pyplot as plt
import ta
import numpy as np
import random
import seaborn as sns
from utilities.backtesting import plot_wallet_vs_asset, get_metrics, get_n_columns, basic_multi_asset_backtest, plot_sharpe_evolution, plot_bar_by_month
from utilities.custom_indicators import SuperTrend
pd.options.mode.chained_assignment = None  # default='warn'


In [2]:
import tensorflow as tf
# tf.test.is_gpu_available(
#     cuda_only=False, min_cuda_compute_capability=None
# )
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
Binance_USDT_HALAL = [
    "BTC/USDT",
    "LUNA/USDT",
    "ETH/USDT",
    "GMT/USDT",
    "UST/USDT",
    "SOL/USDT",
    "APE/USDT",
    "XRP/USDT",
    "IDEX/USDT",
    "AVAX/USDT",
    "DOT/USDT",
    "ADA/USDT",
    "JASMY/USDT",
    "TRX/USDT",
    "NEAR/USDT",
    "AXS/USDT",
    "GAL/USDT",
    "GALA/USDT",
    "SHIB/USDT",
    "ZIL/USDT",
    "ENS/USDT",
    "DOGE/USDT",
    "LTC/USDT",
    "EUR/USDT",
    "MANA/USDT",
    "DAR/USDT",
    "WAVES/USDT",
    "LAZIO/USDT",
    "ALICE/USDT",
    "ROSE/USDT",
    "ZEC/USDT",
    "ALGO/USDT",
    "GRT/USDT",
    "PSG/USDT",
    "SLP/USDT",
    "EOS/USDT",
    "PORTO/USDT",
    "ICP/USDT",
    "EGLD/USDT",
    "XMR/USDT",
    "KDA/USDT",
    "ETC/USDT",
    "MBOX/USDT",
    "OGN/USDT",
    "AR/USDT",
    "GLMR/USDT",
    "LOKA/USDT",
    "XLM/USDT",
    "MTL/USDT",
    "SNX/USDT",
    "PYR/USDT",
    "DASH/USDT",
    "CITY/USDT",
    "ASTR/USDT",
    "IOTA/USDT",
    "ACM/USDT",
    "BAR/USDT",
    "JUV/USDT",
    "SYS/USDT",
    "RVN/USDT",
    "MBL/USDT",
    "REN/USDT",
    "JST/USDT",
    "OMG/USDT",
    "ATM/USDT",
    "XEC/USDT",
    "STORJ/USDT",
    "ZRX/USDT",
    "SRM/USDT",
    "ICX/USDT",
    "API3/USDT",
    "ONT/USDT",
    "SKL/USDT",
    "MULTI/USDT",
    "QTUM/USDT",
    "COCOS/USDT",
    "VOXEL/USDT",
    "HIVE/USDT",
    "KP3R/USDT",
    "ATA/USDT",
    "STMX/USDT",
    "ADX/USDT",
    "HIGH/USDT",
    "NULS/USDT",
    "MLN/USDT",
    "YGG/USDT",
    "SC/USDT",
    "CKB/USDT",
    "TOMO/USDT",
    "STX/USDT",
    "FLUX/USDT",
    "DNT/USDT",
    "ORN/USDT",
    "PLA/USDT",
    "BADGER/USDT",
    "DF/USDT",
    "MOB/USDT",
    "LPT/USDT",
    "SCRT/USDT",
    "RAD/USDT",
    "NMR/USDT",
    "ELF/USDT",
    "TORN/USDT",
    "T/USDT",
    "QUICK/USDT",
    "LSK/USDT",
    "FIDA/USDT",
    "XNO/USDT",
    "BTG/USDT",
    "GHST/USDT",
    "EPS/USDT"
]

pair_list = Binance_USDT_HALAL
tf = '1m'
oldest_pair = "BTC/USDT"
df_list1m = {}
df_list1d = {}
df_list1h = {}
df_list5m = {}
df_list15m = {}


for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1m', path="./database/")
    df_list1m[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1d', path="./database/")
    df_list1d[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1h', path="./database/")
    df_list1h[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '5m', path="./database/")
    df_list5m[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(
        ccxt.binance(), pair, '15m', path="./database/")
    df_list15m[pair] = df.loc[:]
del(df)
df_list = df_list1m
print("Data load 100% use df_list1d[\"BTC/USDT\"] for exemple to access")


Data load 100% use df_list1d["BTC/USDT"] for exemple to access


In [5]:
#df_list1d["BTC/USDT"]


In [101]:
# btc1m["high1"]=btc1m.loc[1:]["high"]

def expand_row(dataframe, window=2):
    df = dataframe.copy()
    for i in range(1, window+1):
        df["high"+str(i)] = df["high"][i:]
        df["low"+str(i)] = df["low"][i:]
        df["open"+str(i)] = df["open"][i:]
        df["close"+str(i)] = df["close"][i:]
        df["volume"+str(i)] = df["volume"][i:]
    return df

def expand_previous(dataframe, window=2):
    df = dataframe.copy()
    for i in range(1, window+1):
        try:
            df.loc[window:len(df),"high-"+str(i)]=None
            df["high-"+str(i)].iloc[window:len(df)]=df["high"][window-i:len(df)-i]

            df.loc[window:len(df),"low-"+str(i)]=None
            df["low-"+str(i)].iloc[window:len(df)]=df["low"][window-i:len(df)-i]

            df.loc[window:len(df),"open-"+str(i)]=None
            df["open-"+str(i)].iloc[window:len(df)]=df["open"][window-i:len(df)-i]            
            
            df.loc[window:len(df),"close-"+str(i)]=None
            df["close-"+str(i)].iloc[window:len(df)]=df["close"][window-i:len(df)-i]            
            
            df.loc[window:len(df),"volume-"+str(i)]=None
            df["volume-"+str(i)].iloc[window:len(df)]=df["volume"][window-i:len(df)-i]
            
            # df["high-"+str(i)][i:] = df["high"][i-1:]
            # df["low-"+str(i)][i:] = df["low"][i-1:]
            # df["open-"+str(i)][i:] = df["open"][i-1:]
            # df["close-"+str(i)][i:] = df["close"][i-1:]
            # df["volume-"+str(i)][i:] = df["volume"][i-1:]
        except:
            print(i)
    return df.iloc[window:]

def expand_timeframe(df_minutes,df_hours, window=2):
    dfm = df_minutes.copy()
    for j in range(1, window+1):
        for i in df_hours[dfm.iloc[0].name:].index:
        #print(str(i))
            try:
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"high_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['high']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"low_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['low']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"open_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['open']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"close_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['close']
            except:
                print("Error Merging: "+str(i))
    
    return dfm


def float_or_not(var):
    try:
        x=float(var)
    except:
        x=None
    return x

def expand_to_1h(df_1m,df_1h, window=2):
    dfm = df_1m.copy()
    index_start=df_1h.index.intersection(dfm.index.round(freq='H'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=60, freq="min"))
                dfm.loc[timefragment,"high_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['high'])
                dfm.loc[timefragment,"low_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['low'])
                dfm.loc[timefragment,"open_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['open'])
                dfm.loc[timefragment,"close_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['close'])
            # except:
            #     print("error fonction "str(i))
    return dfm

def expand_to_4h(df_1m,df_4h, window=2):
    dfm = df_1m.copy()
    #index_start=df_1h[str(dfm.iloc[0].name.round(freq='H')):].index.intersection(dfm.index)
    index_start=df_4h.index.intersection(dfm.index.round(freq='4H'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=4*60, freq="min"))
                dfm.loc[timefragment,"high_4h-"+str(j)]=float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['high'])
                dfm.loc[timefragment,"low_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['low'])
                dfm.loc[timefragment,"open_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['open'])
                dfm.loc[timefragment,"close_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['close'])
            # except:
            #     print("error fonction "str(i))
    return dfm

def expand_to_1d(df_1m,df_1d, window=2,time_suffix="1d"):
    dfm = df_1m.copy()
    index_start=df_1d.index.intersection(dfm.index.round(freq='1d'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                print(i)
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=24*60, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['close'])
            # except:
            #     print("error fonction "str(i))
    return dfm

def expand_to_5m(df_1m,df_5m, window=2,time_suffix="5m"):
    dfm = df_1m.copy()
    index_start=df_5m.index.intersection(dfm.index.round(freq='5 min'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=5, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['close'])
            # except:
            #     print("error fonction "str(i))
    return dfm


def expand_to_15m(df_1m,df_15m, window=2,time_suffix="15m"):
    dfm = df_1m.copy()
    index_start=df_15m.index.intersection(dfm.index.round(freq='5 min'))
    for i in index_start:
        for j in range(1, window+1):    
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=15, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['close'])
            # except:
            #     print("error fonction "str(i))
    return dfm

def rapid1d_expand(df1m,df1d,window=2):
    d1min=df1m.copy()
    d1day=df1d.loc[
    d1min.index[0].round(freq='1d'):
    d1min.index[len(d1min)-1].round(freq='1d')+pd.Timedelta('1 day')
    ].copy()
    d1day_pre=expand_previous(d1day,window)
    d1day_pre=d1day_pre.drop(columns=['open', 'low','close','high','volume'])
    d1day_pre=d1day_pre.add_suffix("_day")
    d1min=pd.merge_asof(
    d1min, d1day_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_day'),
    tolerance=pd.Timedelta('1 day'), allow_exact_matches=True, direction='backward')
    return d1min

def rapid1h_expand(df1m,df1h,window=2):
    d1min=df1m.copy()
    d1hour=df1h.loc[
    d1min.index[0].round(freq='H'):
    d1min.index[len(d1min)-1].round(freq='H')+pd.Timedelta('1 hour')
    ].copy()
    d1hour_pre=expand_previous(d1hour,window)
    d1hour_pre=d1hour_pre.drop(columns=['open', 'low','close','high','volume'])
    d1hour_pre=d1hour_pre.add_suffix("_hour")
    d1min=pd.merge_asof(
    d1min, d1hour_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_hour'),
    tolerance=pd.Timedelta('1 hour'), allow_exact_matches=True, direction='backward')
    return d1min


def rapid5m_expand(df1m,df5m,window=2):
    d1min=df1m.copy()
    d5min=df5m.loc[
    d1min.index[0].round(freq='5 min'):
    d1min.index[len(d1min)-1].round(freq='5 min')+pd.Timedelta('5 min')
    ].copy()
    d5min_pre=expand_previous(d5min,window)
    d5min_pre=d5min_pre.drop(columns=['open', 'low','close','high','volume'])
    d5min_pre=d5min_pre.add_suffix("_5min")
    d1min=pd.merge_asof(
    d1min, d5min_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_5min'),
    tolerance=pd.Timedelta('5 min'), allow_exact_matches=True, direction='backward')
    return d1min

def rapid15m_expand(df1m,df15m,window=2):
    d1min=df1m.copy()
    d15min=df15m.loc[
    d1min.index[0].round(freq='15 min'):
    d1min.index[len(d1min)-1].round(freq='15 min')+pd.Timedelta('15 min')
    ].copy()
    d15min_pre=expand_previous(d15min,window)
    d15min_pre=d15min_pre.drop(columns=['open', 'low','close','high','volume'])
    d15min_pre=d15min_pre.add_suffix("_15min")
    d1min=pd.merge_asof(
    d1min, d15min_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_15min'),
    tolerance=pd.Timedelta('15 min'), allow_exact_matches=True, direction='backward')
    return d1min

def full_expand(df1m,df5m,df15m,df1h,df1d,window=2):
    d1min=df1m.copy()
    d1min=expand_previous(d1min)
    d1min=rapid1d_expand(d1min,df1d,window)
    d1min=rapid1h_expand(d1min,df1h,window)
    d1min=rapid15m_expand(d1min,df15m,window)
    d1min=rapid5m_expand(d1min,df5m,window)
    

    return d1min
def day_expand(data_full):
    ser = pd.to_datetime(pd.Series(data_full.index))
    data_full["day"]=ser.dt.isocalendar().day.values
    data_full["hour"]=ser.dt.hour.values
    data_full["minute"]=ser.dt.minute.values

# merging
def pair_btc(pair="LTC/USDT",window=2):
    Pair_Full=full_expand(df_list1m[pair],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[df_list1m[pair].iloc[0].name:
        df_list1m[pair].iloc[len(df_list1m[pair])-1].name],
        df_list5m["BTC/USDT"],df_list15m["BTC/USDT"],df_list1h["BTC/USDT"],df_list1d["BTC/USDT"],window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='outer',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    return Merged




In [5]:
# def expand_to_1h(df_1m,df_1h, window=2):
#     dfm = df_1m.copy()
#     #index_start=df_1h[str(dfm.iloc[0].name.round(freq='H')):].index.intersection(dfm.index)
#     index_start=df_1h.index.intersection(dfm.index.round(freq='H'))
#     for j in range(1, window+1):
#         for i in index_start:
#             # try:    
#                 timefragment=dfm.index.intersection(pd.date_range(str(i), periods=60, freq="min"))
#                 #print(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['high'].values[0])
#                 dfm.loc[timefragment,"high_1h-"+str(j)]=df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['high'].values[0]
#                 dfm.loc[timefragment,"low_1h-"+str(j)]= df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['low'].values[0]
#                 dfm.loc[timefragment,"open_1h-"+str(j)]= df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['open'].values[0]
#                 dfm.loc[timefragment,"close_1h-"+str(j)]= df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['close'].values[0]
#             # except:
#             #     print("error fonction "str(i))
#     return dfm


In [102]:
#btc1m = expand_previous(df_list1m["BTC/USDT"].iloc[0:10000], 3)
data_full=pair_btc(pair="LTC/USDT",window=10)

c:\LIB\Anaconda3\lib\site-packages\pandas\core\indexing.py:719: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  indexer = self._get_setitem_indexer(key)


In [109]:
data_full[data_full.isnull().any(axis=1)]


,open,high,low,close,volume,high-1,low-1,open-1,close-1,volume-1,...,BTC_open-9_5min,BTC_close-9_5min,BTC_volume-9_5min,BTC_high-10_5min,BTC_low-10_5min,BTC_open-10_5min,BTC_close-10_5min,BTC_volume-10_5min,day,hour
date,,,,,,,,,,,,,,,,,,,,,
2020-07-01 00:02:00,41.07,41.07,40.96,41.00,952.62714,41.15,41.07,41.14,41.07,264.70116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
2020-07-01 00:03:00,40.98,41.04,40.98,41.02,19.45277,41.07,40.96,41.07,41.00,952.62714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
2020-07-01 00:04:00,41.02,41.04,41.02,41.04,111.24215,41.04,40.98,40.98,41.02,19.45277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
2020-07-01 00:05:00,41.04,41.04,40.99,40.99,478.42388,41.04,41.02,41.02,41.04,111.24215,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
2020-07-01 00:06:00,40.99,41.03,40.97,41.03,52.08284,41.04,40.99,41.04,40.99,478.42388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-18 19:59:00,68.50,68.50,68.30,68.50,483.86100,68.50,68.30,68.40,68.50,778.81300,...,29401.93,29288.05,271.30081,29538.19,29393.6,29439.15,29401.94,498.08957,3,19
2022-05-18 20:00:00,68.40,68.50,68.30,68.30,447.32600,68.50,68.30,68.50,68.50,483.86100,...,29288.04,29277.89,202.95460,29401.94,29257.4,29401.93,29288.05,271.30081,3,20
2022-05-18 20:01:00,68.30,68.40,68.30,68.40,13.85900,68.50,68.30,68.40,68.30,447.32600,...,29288.04,29277.89,202.95460,29401.94,29257.4,29401.93,29288.05,271.30081,3,20


In [121]:
data_full

,open,high,low,close,volume,high-1,low-1,open-1,close-1,volume-1,...,BTC_open-9_5min,BTC_close-9_5min,BTC_volume-9_5min,BTC_high-10_5min,BTC_low-10_5min,BTC_open-10_5min,BTC_close-10_5min,BTC_volume-10_5min,day,hour
date,,,,,,,,,,,,,,,,,,,,,
2020-07-01 00:02:00,41.07,41.07,40.96,41.00,952.62714,41.15,41.07,41.14,41.07,264.70116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
2020-07-01 00:03:00,40.98,41.04,40.98,41.02,19.45277,41.07,40.96,41.07,41.00,952.62714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
2020-07-01 00:04:00,41.02,41.04,41.02,41.04,111.24215,41.04,40.98,40.98,41.02,19.45277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
2020-07-01 00:05:00,41.04,41.04,40.99,40.99,478.42388,41.04,41.02,41.02,41.04,111.24215,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
2020-07-01 00:06:00,40.99,41.03,40.97,41.03,52.08284,41.04,40.99,41.04,40.99,478.42388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-18 19:59:00,68.50,68.50,68.30,68.50,483.86100,68.50,68.30,68.40,68.50,778.81300,...,29401.93,29288.05,271.30081,29538.19,29393.6,29439.15,29401.94,498.08957,3,19
2022-05-18 20:00:00,68.40,68.50,68.30,68.30,447.32600,68.50,68.30,68.50,68.50,483.86100,...,29288.04,29277.89,202.95460,29401.94,29257.4,29401.93,29288.05,271.30081,3,20
2022-05-18 20:01:00,68.30,68.40,68.30,68.40,13.85900,68.50,68.30,68.40,68.30,447.32600,...,29288.04,29277.89,202.95460,29401.94,29257.4,29401.93,29288.05,271.30081,3,20


In [118]:
#ltc1m = expand_row(df_list1m["LTC/USDT"], 3)
sys.getsizeof(data_full)/(1024**2)

3292.318660736084

In [ ]:
df_list1h["BTC/USDT"].loc["2022-05-19 15"]

,open,high,low,close,volume
date,,,,,
2022-05-19 15:00:00,29867.7,30545.18,29802.72,30389.45,7993.23976
